In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import string
import re
%matplotlib inline

In [2]:
#veritabanının eklenmesi
df = pd.DataFrame()
df = pd.read_csv("hb.csv")
df.columns =['Rating', 'Review', 'URL']
#df = df.head(10000)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272216 entries, 0 to 272215
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Rating  272216 non-null  int64 
 1   Review  272216 non-null  object
 2   URL     272216 non-null  object
dtypes: int64(1), object(2)
memory usage: 6.2+ MB


In [3]:
df

,Rating,Review,URL
0,5,3 yıldır tık demedi. :),https://www.hepsiburada.com/logitech-m175-kabl...
1,5,3 yıldır kullanıyorum müthiş,https://www.hepsiburada.com/logitech-m175-kabl...
2,4,Ürün bugün elime geçti çok fazla inceleme fırs...,https://www.hepsiburada.com/logitech-m175-kabl...
3,4,Almaya karar verdim. Hemencecik geldi. Keyifle...,https://www.hepsiburada.com/logitech-m175-kabl...
4,5,Günlük kullanımınızı çok çok iyi karsılıyor kı...,https://www.hepsiburada.com/logitech-m175-kabl...
...,...,...,...
272211,5,fiyatına göre güzel,https://www.hepsiburada.com/samsung-galaxy-gra...
272212,5,Ürün kullanışlı iş görüyor fazlasıyla eşime al...,https://www.hepsiburada.com/samsung-galaxy-gra...
272213,5,"Hızlı Kargo, güzel ürün",https://www.hepsiburada.com/samsung-galaxy-gra...
272214,5,telefon başarılı hızlı bir cihaz sadece beyaz...,https://www.hepsiburada.com/samsung-galaxy-gra...


In [4]:
def mentionRemove(text):
    text = re.sub(r'@\w+', '', text)#mentionların çıkartırılması
    return text

In [5]:
for i in range(0,len(df['Review'])):
    text = df['Review'][i]
    text = text.lower()
    text = mentionRemove(text)
    df.at[i,'Review'] = text

In [6]:
def siteRemove(text):
    text = re.sub(r'http.?://[^\s]+[\s]?', '', text)#bağlantıların çıkarılması
    return text

In [7]:
for i in range(0,len(df['Review'])):
    text = df['Review'][i]
    text = siteRemove(text)
    df.at[i,'Review'] = text

In [8]:
def puncRemove(text):
    text = re.sub('[^a-zığüşiöç\s]', '', text)#özel karakterlerin ve sayıların çıkarılması
    text = re.sub(r"Â", "A", text)#umlautlu karakterlerinn değiştirilmesi
    text = re.sub(r"â", "a", text)
    text = re.sub(r"Î", "I", text)
    text = re.sub(r"î", "ı", text)
    text = re.sub(r"Û", "U", text)
    text = re.sub(r"û", "u", text)

    return text

In [9]:
for i in range(0,len(df['Review'])):
    text = df['Review'][i]
    text = puncRemove(text)
    df.at[i,'Review'] = text

In [10]:
def emojiRemove(text):
    # Smile -- :), : ), :-), (:, ( :, (-:, :')
    text = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', ' EMO_POS ', text)
    # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    text = re.sub(r'(:\s?D|:-D|x-?D|X-?D)', ' EMO_POS ', text)
    # Love -- <3, :*
    text = re.sub(r'(<3|:\*)', ' EMO_POS ', text)
    # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
    text = re.sub(r'(;-?\)|;-?D|\(-?;)', ' EMO_POS ', text)
    # Sad -- :-(, : (, :(, ):, )-:
    text = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', ' EMO_NEG ', text)
    # Cry -- :,(, :'(, :"(
    text = re.sub(r'(:,\(|:\'\(|:"\()', ' EMO_NEG ', text)
    return text

In [11]:
# java virtual machine kullanılarak zemberek uygulaması ile tokenization ve stemming yapıldı
import zemberek
from jpype import JClass, JString, getDefaultJVMPath, shutdownJVM, startJVM, java
#Zemberek kütüphanesinin kullanılabilmesi için Jpype ile JVM oluşturulması
ZEMBEREK_PATH = r'zemberek-full.jar'
startJVM(getDefaultJVMPath(), '-ea', '-Djava.class.path=%s' % (ZEMBEREK_PATH))
#Morphology kütüphanesini kullanacağımızdan burada oluşturuyoruz
TurkishMorphology = JClass('zemberek.morphology.TurkishMorphology')
morphology = TurkishMorphology.createWithDefaults()

# java virtual machine kullanılarak zemberek uygulaması ile tokenization ve stemming yapıldı
def stem_vm(text):
    arr=[]
    textlist=text.split(" ")
    if(text=='        '):
        text='bos'
    analysis: java.util.ArrayList = ( morphology.analyzeAndDisambiguate(text).bestAnalysis() )
    for i, analysis in enumerate(analysis, start=1):
        f'\nAnalysis {i}: {analysis}',
        f'\nPrimary POS {i}: {analysis.getPos()}' 
        arr.append(f'{str(analysis.getLemmas()[0])}')
    return arr

In [12]:
def rootFinder(text):
    pos=[]
    textlist=text.split(" ")
    analysis: java.util.ArrayList = ( morphology.analyzeAndDisambiguate(text).bestAnalysis() )
    for i, analysis in enumerate(analysis, start=1):
        f'\nAnalysis {i}: {analysis}',
        f'\nPrimary POS {i}: {analysis.getPos()}' 
        if (str(analysis.getLemmas()[0]) == "UNK"):
            pos.append(textlist[i-1])
        else:     
            pos.append(f'{str(analysis.getLemmas()[0])}')
    return pos

In [13]:
for i in range(0,len(df['Review'])):
    text = df['Review'][i]
    text = rootFinder(text)
    text=' '.join(text)
    df.at[i,'Review'] = text

In [14]:
from nltk.tokenize import word_tokenize

stopwords = np.loadtxt("stopwords.txt", dtype=str)
def stopwords_sil(text):
    text_tokenize = word_tokenize(text)
    text = [word for word in text_tokenize if not word in stopwords]
    return text

In [15]:
for i in range(0,len(df['Review'])):
    text = df['Review'][i]
    text = stopwords_sil(text)
    text=' '.join(text)
    df.at[i,'Review'] = text

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272216 entries, 0 to 272215
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Rating  272216 non-null  int64 
 1   Review  272216 non-null  object
 2   URL     272216 non-null  object
dtypes: int64(1), object(2)
memory usage: 6.2+ MB


In [18]:
df

,Rating,Review,URL
0,5,yıl tık de,https://www.hepsiburada.com/logitech-m175-kabl...
1,5,yıl kullan müthiş,https://www.hepsiburada.com/logitech-m175-kabl...
2,4,ürün bugün el geç çok fazla incele fırsat ol g...,https://www.hepsiburada.com/logitech-m175-kabl...
3,4,al karar ver hemencecik gel keyif kullan,https://www.hepsiburada.com/logitech-m175-kabl...
4,5,günlük kullanım çok çok iyi karsılıyor kısaç m...,https://www.hepsiburada.com/logitech-m175-kabl...
...,...,...,...
272211,5,fiyat göre güzel,https://www.hepsiburada.com/samsung-galaxy-gra...
272212,5,ürün kullan iş gör fazlasıyla eş aldi cok memn...,https://www.hepsiburada.com/samsung-galaxy-gra...
272213,5,hız kargo güzel ürün,https://www.hepsiburada.com/samsung-galaxy-gra...
272214,5,telefon başarı hız cihaz beyaz iste gri renk gel,https://www.hepsiburada.com/samsung-galaxy-gra...
